In [1]:
def GPS_Failure_Risk_Calc(SatStatus=None, time=None, Lambda = None, ):

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        # Program Name : Markov-based Drone's Reliability and MTTF Estimator      %
        # Author       : Koorosh Aslansefat                                       %
        # Version      : 1.0.0                                                    %
        # Description  : A Markov Process-Based Approach for Reliability          %
        #                Evaluation of the GPS System for Drones                  %
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        '''
        
        SatStatus: is an integer between 0 and 11. 0 means no satellite is available and 14 means fully satellite coverage. 
        
        Lambda: Failure Rate of the GPS System.
        
        time: Time of the mission
        '''
        
        import numpy as np  # linear algebra
        import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
        import sympy as sym # Symbolic Calculation
        
        t = sym.Symbol('t')

        L = Lambda
        
        if SatStatus == 14:
            P0_GPS = sym.Matrix([[1],[0],[0],[0],[0],[0],[0],[0],[0]])
            Sflag = 7
        elif SatStatus == 13:
            P0_GPS = sym.Matrix([[0],[1],[0],[0],[0],[0],[0],[0],[0]])
            Sflag = 6
        elif SatStatus == 12:
            P0_GPS = sym.Matrix([[0],[0],[1],[0],[0],[0],[0],[0],[0]])
            Sflag = 5
        elif SatStatus == 11:
            P0_GPS = sym.Matrix([[0],[0],[0],[1],[0],[0],[0],[0],[0]])
            Sflag = 4
        elif SatStatus == 10:
            P0_GPS = sym.Matrix([[0],[0],[0],[0],[1],[0],[0],[0],[0]])
            Sflag = 3
        elif SatStatus == 9:
            P0_GPS = sym.Matrix([[0],[0],[0],[0],[0],[1],[0],[0],[0]])
            Sflag = 2
        elif SatStatus == 8:
            P0_GPS = sym.Matrix([[0],[0],[0],[0],[0],[0],[1],[0],[0]])
            Sflag = 1
        elif SatStatus == 7:
            P0_GPS = sym.Matrix([[0],[0],[0],[0],[0],[0],[0],[1],[0]])
            Sflag = 0
        else:
            P_Fail = 1
            MTTF = 0

        M_GPS = sym.Matrix([[-14*L,     0,     0,     0,     0,     0,     0,     0,    0],
                            [ 14*L, -13*L,     0,     0,     0,     0,     0,     0,    0],
                            [    0,  13*L, -12*L,     0,     0,     0,     0,     0,    0],
                            [    0,     0,  12*L, -11*L,     0,     0,     0,     0,    0],
                            [    0,     0,     0,  11*L, -10*L,     0,     0,     0,    0],
                            [    0,     0,     0,     0,  10*L,  -9*L,     0,     0,    0],
                            [    0,     0,     0,     0,     0,   9*L,  -8*L,     0,    0],
                            [    0,     0,     0,     0,     0,     0,   8*L,  -7*L,    0],
                            [    0,     0,     0,     0,     0,     0,     0,   7*L,    0]])

        P_GPS = sym.exp(M_GPS*t)*P0_GPS

        P_GPS_Fail = P_GPS[-1]

        N_GPS = sym.Matrix([[-14*L,     0,     0,     0,     0,     0,     0,     0],
                            [ 14*L, -13*L,     0,     0,     0,     0,     0,     0],
                            [    0,  13*L, -12*L,     0,     0,     0,     0,     0],
                            [    0,     0,  12*L, -11*L,     0,     0,     0,     0],
                            [    0,     0,     0,  11*L, -10*L,     0,     0,     0],
                            [    0,     0,     0,     0,  10*L,  -9*L,     0,     0],
                            [    0,     0,     0,     0,     0,   9*L,  -8*L,     0],
                            [    0,     0,     0,     0,     0,     0,   8*L,  -7*L]])
        
        tt = -1*N_GPS.inv()
        
        MTTF_GPS = sum(tt[Sflag,:])

        return P_GPS_Fail.evalf(subs={t: time}), MTTF_GPS.evalf(subs={t: time})

In [2]:
GPS_P_Fail, GPS_MTTF = GPS_Failure_Risk_Calc(SatStatus=14, time=100, Lambda = 0.001)
print(GPS_P_Fail)
print(GPS_MTTF)

1.19038762535297e-5
1142.85714285714


In [3]:
GPS_P_Fail, GPS_MTTF = GPS_Failure_Risk_Calc(SatStatus=10, time=100, Lambda = 0.001)
print(GPS_P_Fail)
print(GPS_MTTF)

0.0107555729878382
363.636363636364


In [4]:
import sympy as sym

L = sym.symbols('L') 

M_GPS_updated = sym.zeros(22, 22)

start_value = -28

for i in range(22):
    if i == 21:
        M_GPS_updated[i, i] = (start_value + i) * L
    else:
        M_GPS_updated[i, i] = (start_value + i) * L
        M_GPS_updated[i+1, i] = -(start_value + i) * L  

sym.pprint(M_GPS_updated)

⎡-28⋅L    0      0      0      0      0      0      0      0      0      0    
⎢                                                                             
⎢28⋅L   -27⋅L    0      0      0      0      0      0      0      0      0    
⎢                                                                             
⎢  0    27⋅L   -26⋅L    0      0      0      0      0      0      0      0    
⎢                                                                             
⎢  0      0    26⋅L   -25⋅L    0      0      0      0      0      0      0    
⎢                                                                             
⎢  0      0      0    25⋅L   -24⋅L    0      0      0      0      0      0    
⎢                                                                             
⎢  0      0      0      0    24⋅L   -23⋅L    0      0      0      0      0    
⎢                                                                             
⎢  0      0      0      0      0    23⋅L   -22⋅L    

In [5]:
def GPS_Failure_Risk_Calc2(SatStatus=None, time=None, Lambda = None, MaxSat = 28, MinSat = 7):

        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        # Program Name : Markov-based Drone's Reliability and MTTF Estimator      %
        # Author       : Koorosh Aslansefat                                       %
        # Version      : 1.0.5                                                    %
        # Description  : A Markov Process-Based Approach for Reliability          %
        #                Evaluation of the GPS System for Drones                  %
        #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        '''
        
        SatStatus: is an integer between 0 and MaxSat. 0 means no satellite is available and 14 means fully satellite coverage. 
        
        Lambda: Failure Rate of the GPS System.
        
        time: Time of the mission
        '''
        
        import numpy as np  # linear algebra
        import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
        import sympy as sym # Symbolic Calculation
        
        t = sym.Symbol('t')

        L = Lambda
        
        if 7 <= SatStatus <= 28:
            P0_GPS = sym.zeros(22, 1)
            P0_GPS[SatStatus - 7] = 1
            Sflag = 28 - SatStatus
        else:
            P_Fail = 1
            MTTF = 0
        
        M_GPS_updated = sym.zeros(MaxSat-MinSat+1, MaxSat-MinSat+1)
        start_value = -1*MaxSat
        
        for i in range(MaxSat-MinSat+1):
            if i == 21:
                M_GPS_updated[i, i] = (start_value + i) * L
            else:
                M_GPS_updated[i, i] = (start_value + i) * L
                M_GPS_updated[i+1, i] = -(start_value + i) * L 
                
        M_GPS = sym.Matrix(M_GPS_updated)
        
        P_GPS = sym.exp(M_GPS*t)*P0_GPS

        P_GPS_Fail = P_GPS[-1]
        
        N_GPS = M_GPS[:-1, :-1] 
        
        tt = -1*N_GPS.inv()
        print(Sflag)
        MTTF_GPS = sum(tt[Sflag,:])

        return P_GPS_Fail.evalf(subs={t: time}), MTTF_GPS.evalf(subs={t: time})
        

In [6]:
import  sys
import os
from sys import platform

#from temp import SafeDrones

if platform == "linux" or platform == "linux2":
    currentdir = os.path.dirname(os.path.realpath(inspect.getfile(inspect.currentframe())))
    parentdir = os.path.dirname(currentdir)
    sys.path.insert(0, parentdir)
elif platform == "win32":
    # in Windows, the first path entry contains the directory where the notebook is
    sys.path.insert(0, os.path.dirname(sys.path[0]))
from SafeDrones.core.SafeDrones import SafeDrones
eval = SafeDrones()

In [7]:
GPS_P_Fail, GPS_MTTF = eval.GPS_Failure_Risk_Calc(SatStatus=27, time=100, Lambda = 0.001, MaxSat = 29, MinSat = 17)
print(GPS_P_Fail)
print(GPS_MTTF)

0.282896728355594
111.111111111111


In [8]:
GPS_P_Fail, GPS_MTTF = eval.GPS_Failure_Risk_Calc(SatStatus=29, time=100, Lambda = 0.001, MaxSat = 29, MinSat = 17)
print(GPS_P_Fail)
print(GPS_MTTF)

0.182683524052735
34.4827586206897
